In [1]:
%run functions.ipynb

Ready: 2023-05-03 00:39:52


In [2]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()
spark

In [16]:
df_new = spark.read.parquet(f'files/ten_city/df_dates')
df_new.createOrReplaceTempView('df_new')

In [18]:
def count_dates_all(col, df_new='df_new'):
    dff1 = spark.sql(f'''

    with t as
        (
        select 
            year_r
            , city
            , {col}
            , count(*) cnt
        from {df_new}
        group by
            year_r
            , city
            , {col}
        order by
            year_r
            , city
            , {col}
        )
    ,gr as
        (
        select
            {col} 
            ,max(case when city = 'Астана'  then cnt end) cnt_a
            ,max(case when city = 'Бишкек'  then cnt end) cnt_b
            ,max(case when city = 'Москва'  then cnt end) cnt_m
            ,max(case when city = 'ОАЭ'     then cnt end) cnt_o
            ,max(case when city = 'Ташкент' then cnt end) cnt_t
            
            ,max(case when city = 'СПБ'  then cnt end) cnt_s
            ,max(case when city = 'Новосибирск'  then cnt end) cnt_n
            ,max(case when city = 'Владивосток'  then cnt end) cnt_v
            ,max(case when city = 'Алматы'     then cnt end) cnt_all
            ,max(case when city = 'Калининград' then cnt end) cnt_k
        from t
        group by
            {col}
        order by
            {col}    
        )

    select
        {col}
        ,sum(cnt_a) over(partition by {col}) / sum(cnt_a) over()*100 cnt_a
        ,sum(cnt_b) over(partition by {col}) / sum(cnt_b) over()*100 cnt_b
        ,sum(cnt_m) over(partition by {col}) / sum(cnt_m) over()*100 cnt_m
        ,sum(cnt_o) over(partition by {col}) / sum(cnt_o) over()*100 cnt_o
        ,sum(cnt_t) over(partition by {col}) / sum(cnt_t) over()*100 cnt_t
        
        ,sum(cnt_s) over(partition by {col}) / sum(cnt_s) over()*100 cnt_s
        ,sum(cnt_n) over(partition by {col}) / sum(cnt_n) over()*100 cnt_n
        ,sum(cnt_v) over(partition by {col}) / sum(cnt_v) over()*100 cnt_v
        ,sum(cnt_all) over(partition by {col}) / sum(cnt_all) over()*100 cnt_all
        ,sum(cnt_k) over(partition by {col}) / sum(cnt_k) over()*100 cnt_k
    from gr

    ''').toPandas()
    return dff1

In [19]:
df_new_list = []
for i in tqdm(['hour_r','date_r','week_r','week_day_r']):
    df_new_list.append(count_dates_all(i))

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [09:50<00:00, 147.70s/it]


In [21]:
dff1 = df_new_list[0]
dff2 = df_new_list[1]
dff3 = df_new_list[2]
dff4 = df_new_list[3]

In [23]:
t = ['Час','Количество запросов Астана','Количество запросов Бишкек','Количество запросов Москва','Количество запросов ОАЭ','Количество запросов Ташкент', 'Количество запросов СПБ', 'Количество запросов Новосибирск'
    , 'Количество запросов Владимир', 'Количество запросов Алматы', 'Количество запросов Калининград']


t[0] = 'Час'
dff1.columns = t
t[0] = 'Дата'
dff2.columns = t
t[0] = 'Неделя'
dff3.columns = t
t[0] = 'День недели'
dff4.columns = t

with pd.ExcelWriter(f'files/ten_city/df_dates_new.xlsx') as writer:
    dff1.to_excel(writer, sheet_name='Часы', index=False)
    dff2.to_excel(writer, sheet_name='Даты', index=False)
    dff3.to_excel(writer, sheet_name='Недели', index=False)
    dff4.to_excel(writer, sheet_name='Дни недели', index=False)

In [42]:
make_linear_plots(f'files/ten_city/df_dates_new.xlsx')

files/ten_city/df_dates_new_report_230503_0159.docx


In [41]:
def make_linear_plots(input_filename, cnt=9999, size_x=17.5, size_y=10.8, font_size=16, img_w=17.5, img_h=10.8, blank=False, word_sample='files/report.docx'):
    def zero(x,y):
        if x < 0:
            return str(0) +'%'
        return str(round(x,1)) +'%'
    
    dt_now = str(datetime.datetime.now())[2:-9].replace('-','').replace(':','').replace(' ','_')
    word_file = f'{input_filename[:-5]}_report_{dt_now}.docx'
    print(word_file)
    #cols1 = ['Час','Количество запросов Астана','Количество запросов Бишкек','Количество запросов Москва','Количество запросов ОАЭ','Количество запросов Ташкент']
    #cols2 = ['Дата','Количество запросов Астана','Количество запросов Бишкек','Количество запросов Москва','Количество запросов ОАЭ','Количество запросов Ташкент']
    #cols3 = ['Неделя','Количество запросов Астана','Количество запросов Бишкек','Количество запросов Москва','Количество запросов ОАЭ','Количество запросов Ташкент']
    #cols4 = ['День недели','Количество запросов Астана','Количество запросов Бишкек','Количество запросов Москва','Количество запросов ОАЭ','Количество запросов Ташкент']
    
    cols1 = ['Час', 'Количество запросов СПБ', 'Количество запросов Новосибирск', 'Количество запросов Владимир', 'Количество запросов Алматы', 'Количество запросов Калининград']
    cols2 = ['Дата', 'Количество запросов СПБ', 'Количество запросов Новосибирск', 'Количество запросов Владимир', 'Количество запросов Алматы', 'Количество запросов Калининград']
    cols3 = ['Неделя', 'Количество запросов СПБ', 'Количество запросов Новосибирск', 'Количество запросов Владимир', 'Количество запросов Алматы', 'Количество запросов Калининград']
    cols4 = ['День недели', 'Количество запросов СПБ', 'Количество запросов Новосибирск', 'Количество запросов Владимир', 'Количество запросов Алматы', 'Количество запросов Калининград']
    dff1 = pd.read_excel(input_filename, sheet_name='Часы')[cols1]
    dff2 = pd.read_excel(input_filename, sheet_name='Даты')[cols2]
    dff3 = pd.read_excel(input_filename, sheet_name='Недели')[cols3]
    dff4 = pd.read_excel(input_filename, sheet_name='Дни недели')[cols4]
    
    dff1_test = dff1.set_index(dff1.columns[0])
    dff2_test = dff2.set_index(dff2.columns[0])
    dff3_test = dff3.set_index(dff3.columns[0])
    dff4_test = dff4.set_index(dff4.columns[0])
    dff4_test.index = (['пн','вт','ср','чт','пт','сб','вс'])
    j=0
    for d in [dff1_test, dff2_test, dff3_test, dff4_test]: 
        j+=1
        fig, ax = plt.subplots()
        ax = sns.lineplot(data = d, linewidth=3, palette=['#4F81BD', '#F79646', '#C0504D', '#9BBB59', '#264478'])
        plt.rcParams['figure.figsize']=17,10.8
        plt.rcParams['font.size'] = '16'
        plt.rcParams['figure.dpi'] = 2**8
        plt.rcParams['figure.facecolor'] = 'white'
        fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")


        #plt.legend(['Астана', 'Бишкек', 'Москва', 'ОАЭ', 'Ташкент'], prop=fpath)
        plt.legend(['СПБ','Новосибирск','Владимир','Алматы','Калининград'], prop=fpath)
        plt.xticks(font=fpath)
        plt.yticks(font=fpath)
        plt.grid(visible=True, linewidth=0.3)
        ax.set_xlabel('')
        ax.yaxis.set_major_formatter(FuncFormatter(zero))
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1)) 
        ax.yaxis.set_major_locator(ticker.LinearLocator(numticks=6))  
        img_file_name = input_filename.replace('files/','')[:-5]
        img_file_name_final = f'pictures/{img_file_name}_{j}.png'
        if 'Час' == d.index.name:
            ax.set_title(f'Запросы по часам, шт.', font=fpath)
        elif 'Дата' == d.index.name:
            ax.xaxis.set_major_locator(ticker.MultipleLocator(4)) 
            ax.set_title(f'Запросы по датам, шт.', font=fpath)
        elif 'Неделя' == d.index.name:
            ax.set_title(f'Запросы по неделям, шт.', font=fpath)
        else:
            ax.set_title(f'Запросы по дням недели, шт.', font=fpath) 

        #im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_2гис_232_354.png")
        #ax.figure.figimage(im, 3250, 150, zorder=1, alpha=0.5)
#
        #im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_шт_232_354.png")
        #ax.figure.figimage(im, 200, 1800, zorder=1, alpha=0.5)

        ax.figure.savefig(img_file_name_final, bbox_inches='tight')
        add_image(word_file, img_file_name_final, '', img_w=img_w, img_h=img_h, blank=blank,word_sample=word_sample)
        #plt.show()
        plt.close()